In [1]:
import json
import requests
from bs4 import BeautifulSoup
import time
import pickle
from utils import *
# from newsapi import NewsApiClient # !pip install newsapi-python

In [2]:
API_KEY = "b942a2d2391e4ad8ae01c9168a5cdcc1" # salbro@mit.edu
API_KEY = "90493a497ace4b8ebe8ef5e82bfadce6" # cyrille's API key
# API_KEY = "f604134c17ae431591a2976d7c3bae55" # stephenpalbro@gmail.com api key

Let's create a massive NBA query consisting of "NBA" + all teams + top 25 players.

In [3]:
## get all NBA teams, add them to the query
with open("teams.txt", "r") as fp:
    teams = ["(" + team.replace("\n", "").strip().lower() + ")" for team in fp.readlines()]

query = "NBA OR " + " OR ".join(teams)
query += " OR "

# add all players to the query
query += " OR ".join(["("+name+")" for name in TOP_100_PLAYERS[:25]]) # top 25 players

# URL-ify the query
query = urlify(query)

#### Let's get the articles now.

In [5]:
%%time
param_dict = {"from": "2017-10-17", "sources":"bleacher-report",
              "pageSize":"100", "apiKey":API_KEY, "q":query} # you can also do "sortBy=popularity"
set_of_docs = get_docs_from_params(param_dict)

0/3272...50/3272...100/3272...150/3272...200/3272...250/3272...300/3272...350/3272...400/3272...450/3272...500/3272...550/3272...600/3272...650/3272...700/3272...750/3272...800/3272...850/3272...900/3272...950/3272...1000/3272...1050/3272...1100/3272...1150/3272...1200/3272...1250/3272...1300/3272...1350/3272...1400/3272...1450/3272...1500/3272...1550/3272...1600/3272...1650/3272...1700/3272...1750/3272...1800/3272...1850/3272...1900/3272...1950/3272...2000/3272...2050/3272...2100/3272...2150/3272...2200/3272...2250/3272...2300/3272...2350/3272...2400/3272...2450/3272...2500/3272...2550/3272...2600/3272...2650/3272...2700/3272...2750/3272...2800/3272...2850/3272...2900/3272...2950/3272...3000/3272...3050/3272...3100/3272...3150/3272...3200/3272...3250/3272...CPU times: user 2min 40s, sys: 9.85 s, total: 2min 50s
Wall time: 34min 17s


In [6]:
big_filename = "all_nba_from_oct17"
with open(big_filename, 'wb') as fp:
    pickle.dump(set_of_docs, fp)

In [24]:
# with open (big_filename, 'rb') as fp:
#     docs = pickle.load(fp)

# players only

In [18]:
player_queries = ["(lebron james) OR lebron", "(kevin durant) OR KD", "(steph curry) OR (stephen curry)", 
                 "(james harden) OR (harden)", "(russel westbrook) OR (westbrook)", "(anthony davis)"]
player_docs_list = {}

start_date = "2017-10-17" # start of 2017-2018 NBA season
for player, name in zip(player_queries, player_names):
    print(name)
    param_dict = {"from": start_date, "sources":"bleacher-report",
              "pageSize":"100", "apiKey":API_KEY, "q":player} # you can also do "sortBy=popularity"

    player_docs = get_docs_from_params(param_dict)
    player_docs_list[player] = player_docs

# save to a file
player_names = ["bron", "kd", "steph", "harden", "russel", "davis"]
for player, name in zip(player_queries, player_names):
    filename = name + "_from_oct17"
    with open(filename, 'wb') as fp:
        pickle.dump(player_docs_list[player], fp)

(lebron james) OR lebron
0/845...50/845...100/845...150/845...200/845...250/845...300/845...350/845...400/845...450/845...500/845...550/845...600/845...650/845...700/845...750/845...800/845...(kevin durrant) OR KD
0/30...(steph curry) OR (stephen curry)
0/351...50/351...100/351...150/351...200/351...250/351...300/351...350/351...(james harden) OR (harden)
0/210...50/210...100/210...150/210...200/210...(russel westbrook) OR (westbrook)
0/242...50/242...100/242...150/242...200/242...(anthony davis)
0/201...50/201...100/201...150/201...200/201...

#### old stuff

Now let's scrape the comments ... having trouble with this because of the iframe. forget about it for the time being.

In [185]:
# %%time
# comments_by_doc = []
# for article in all_articles[:10]:
#     comments = []
#     url = article['url']
#     soup = BeautifulSoup(requests.get(url).text, "lxml")
#     content = soup.find_all("span", {"class":"_5mdd"})
#     if content is None:
#         print("no comment with class _5mdd")
#     for comment_span in content:
#         comments.append(comment_span.get_text())
#     comments_by_doc.append(comments)

CPU times: user 394 ms, sys: 36.2 ms, total: 430 ms
Wall time: 848 ms
